
<h2>Fourier Transform Visualization by @LewisErick</h2>

Based on the notebooks in the following repo by **@jacobstallone**
https://github.com/jacobstallone/D3-in-Jupyter-Notebook

Also credits to **@shimizu** for illustrating how to perform transitions over line chart visualizations
https://bl.ocks.org/shimizu/cbd93e1b735939a9ebfd3708c2a860b3

In [1]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [2]:
import numpy as np

In [3]:
HTML('<script src="https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.17/d3.min.js"></script>')

In [4]:
'''
n: Number of data points to represent the input space of our sample function
t: The data points themselves, ranging from -pi to pi.
'''

n = 1000
t = np.linspace(-np.pi, np.pi, n)

In [5]:
'''
The sample function in this notebook is the product of a cos function with an exp: fn
fn_integrand is the integrand of the Fourier Transform i.e. the wave at a particular frequency value
'''

def fn(x):
    return np.cos(2 * np.pi * 3 * x) * np.exp(-np.pi * x * x)

def fn_integrand(x):
    return np.exp(-2 * np.pi * 1j * (freq * x)) * fn(x)

fn_t = np.apply_along_axis(fn, 0, t)

In [6]:
function_data = []

fft_fn_t = np.fft.fft(fn_t)
real_fft_fn_t = 2.0 * np.abs(fft_fn_t/n)
freqs = np.fft.fftfreq(n) * n

positive_freqs = freqs[:n//2]
positive_fft = real_fft_fn_t[:n//2]

mask = np.argsort(positive_freqs, axis=None)

fft_x = positive_freqs[mask]
fft_y = positive_fft[mask]

for t_i, fn_t_i in zip(fft_x, fft_y):
    function_data.append({'x':t_i, 'y':fn_t_i.real})

In [7]:
function_data = function_data[:100]

In [8]:
css_text = '''
circle {
    fill: steelblue;
    stroke: #000;
    }

    rect {
      fill: steelblue;
    }

    rect:hover {
        fill:green;
        }
    
    .chart text {
      fill: white;
      font: 10px sans-serif;
      text-anchor: end;
    }

.tooltip {
  font-family: Helvetica, Arial, sans-serif;
}

.axis path{
  fill:none;
  stroke:black;
  stroke-width: 0.15 px;
  shape-rendering: crispEdges;
}

.axis text{
  font-family: Helvetica, Arial, sans-serif;
  font-size: 12px;
  text-anchor: end;
}

path {
    stroke-width: 1;
    fill: none;
}

line {
    stroke: black;
}

text{font-family: Helvetica, Arial, sans-serif};
'''

In [9]:
js_text_template = Template('''
var w = 800;
var h = 500;


// variables and setting up the svg element
var margin = {top: 20, right: 20, bottom: 100, left: 60},
    width = w - margin.left - margin.right,
    height = h - margin.top - margin.bottom;

var x = d3.scale.linear()
        .range([0, width]);

var y = d3.scale.linear()
        .range([height, 0]);

// the svg element
var mySVG = d3.select("#graph-div")
        .append("svg")
        .attr("width", width + margin.left + margin.right)
        .attr("height", height + margin.top + margin.bottom)
        .append("g")
        .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

var background =
    mySVG.append("rect")
        .attr("width", width + margin.left + margin.right)
        .attr("height", height + margin.top + margin.bottom)
        .attr("fill-opacity", "0")
        .attr("fill", "white")
        .on("mousemove", point)
        .on("mouseover", over)
        .on("mouseleave", leave)
        .on("click", click);

var myLine = mySVG.append("path");
var circle = mySVG.append("circle")
                .attr("r", 4)
                .attr("fill", "rgb(205,23,25)")
                .style("opacity", "0")
                .attr("pointer-events", "none")
                .attr("stroke-width", "2.5")
                .attr("stroke", "white");

var yaxislabel = mySVG.append("text")
                      .attr("transform", "rotate(-90)")
                      .attr("y", 0 - margin.left)
                      .attr("x",0 - (height / 2))
                      .attr("dy", "1em")
                      .style("text-anchor", "middle");

var titleSVG = mySVG.append("text")
        .attr("x", width / 2)
        .attr("y",  0)
        .style("text-anchor", "middle");

var tooltipX = mySVG.append("text")
        .attr("x", 0)
        .attr("y", 0)
        .style("opacity", "0");
        
var tooltipY = mySVG.append("text")
        .attr("x", 0)
        .attr("y", 0)
        .style("opacity", "0");

var xlabelaxis = mySVG.append("g")
    .attr("class", "x axis")
    .attr("transform", "translate(0," + height/2 + ")") //sets the vertical axis in the middle;

var ylabelaxis = mySVG.append("g")
    .attr("class", "y axis");

var xaxislabel = mySVG.append("text")             
                      .attr("transform",
                            "translate(" + (width/2) + " ," + 
                                           (height + margin.top + 20) + ")")
                      .style("text-anchor", "middle")
                      .text("Date");

function point(){
    var pathEl = myLine.node();
    var pathLength = pathEl.getTotalLength();

    var _x = d3.mouse(this)[0];
    var beginning = _x , end = pathLength, target;
    while (true) {
        target = Math.floor((beginning + end) / 2);
        pos = pathEl.getPointAtLength(target);

        if ((target === end || target === beginning) && pos.x !== _x) {
            break;
        }
        if (pos.x > _x){
            end = target;
        }else if(pos.x < _x){
            beginning = target;
        }else{
            break; //position found
        }
    }
    
    // Update tooltip position and values.
    tooltipX.attr("y", pos.y - 16);
    tooltipX.attr("x", pos.x + 16);
    idx = Math.round((pos.x / width) * xValues.length);
    tooltipX.text("x: " + xValues[idx].toFixed(4));
    
    tooltipY.attr("y", pos.y);
    tooltipY.attr("x", pos.x + 16);
    idx = Math.round((pos.x / width) * xValues.length);
    tooltipY.text("y: " + yValues[idx].toFixed(4));
    
    circle
    .attr("opacity", 1)
    .attr("cx", _x)
    .attr("cy", pos.y);
}

var currentFunction = "fft";

function updateYValues(freq) {
    if (currentFunction == "fft") {
        yValues = fftFunction(plotstart, plotrange, stepsize);
    } else if (currentFunction == "integrand") {
        yValues = integrandFunction(plotstart, plotrange, stepsize, freq);
    } else if (currentFunction == "original") {
        yValues = originalFunction(plotstart, plotrange, stepsize);
    }
    xValues = xAxisValues(plotstart, plotrange, stepsize);
}

var freq = 1;

function click() {
    // Update the function to display.
    if (currentFunction == "fft") {
        var _x = d3.mouse(this)[0];
        freq = Math.round((_x/width)*50);
        currentFunction = "integrand";
    } else if (currentFunction == "integrand") {
        currentFunction = "original";
    }
    updateYValues(freq);
    plotLine(xValues, yValues);
}

function over(){
    circle.transition().duration(200).style("opacity", "1");
    tooltipX.style("opacity", "1");
    tooltipY.style("opacity", "1");
}
function leave(){
    circle.transition().duration(200).style("opacity", "0");
    tooltipX.style("opacity", "0");
    tooltipY.style("opacity", "0");
}

var plotstart = -3, 
    stepsize = 0.012, // in use in this script
    plotrange_real = 3,
    plotrange = plotrange_real + stepsize; // adjusted for the "range" method using stepsize as a 3rd parameter

var yValues, xValues; // declares the values

function fftFunction(startinput, stopinput, steprange)
{ 
    var answer = [];
    var answer_data = $python_data;
    answer_data.forEach(function(d) {
        answer.push(+d.y);
    })
    return answer;
};

function integrandFunction(startinput, stopinput, steprange, freq) 
{ 
    return d3.range(startinput, stopinput, steprange).map(function(i) 
    {
        return Math.exp(-2 * Math.PI * (freq * i)) * (Math.cos(2 * Math.PI * 3 * i) * Math.exp(-Math.PI * i * i));
    });
};

function originalFunction(startinput, stopinput, steprange) 
{ 
    return d3.range(-3, 3, steprange).map(function(i) 
    {
        return (Math.cos(2 * Math.PI * 3 * i) * Math.exp(-Math.PI * i * i));
        
    })
};

function fftXAxisValues() {
    var answer = [];
    var answer_data = $python_data;
    answer_data.forEach(function(d) {
        answer.push(+d.x);
    })
    return answer;
}

function xAxisValues(startinput, stopinput, steprange) 
{
    if (currentFunction == "fft") {
        return fftXAxisValues();
    } else {
        return d3.range(-3, 3, steprange).map(function(i) 
        {
            return i;
        })
    }
};

xValues = xAxisValues(plotstart, plotrange, stepsize); // the generates x-values
yValues = fftFunction(); //these are the y-values, the up and down of the sinuscurve

function plotLine(newXValues, newYValues) {
    var title;
    if (currentFunction == "fft") {
        title = "Fourier Transform of Original Function";
        yaxislabel.text("FFT value");
        xaxislabel.text("Frequency (Hz)")
    } else if (currentFunction == "integrand") {
        title = "Integrand of FFT with frequency " + freq; 
        yaxislabel.text("Integrand value"); 
        xaxislabel.text("t")
    } else {
        yaxislabel.text("Original function value"); 
        title = "Original Function";
        xaxislabel.text("t")
    }
    titleSVG
        .transition().duration(1000)
        .text(title);

    // create the domain for the values
    // scale the data to fit in our svg
    var scaleX = d3.scale.linear()
        .domain([d3.min(newXValues), d3.max(newXValues)])
        .range([0, width]);

    var scaleY = d3.scale.linear()
        .domain([d3.min(newYValues), d3.max(newYValues)])
        .range([height, 0]); //remember the order of this one! otherwise you'll get an opposite sinus curve

    // picks out the data for the line
    var line = d3.svg.line()
        .x(function(d) { return scaleX(d.x); }) //we define x and y in the foreach function below (a little unorderly yes, admitted)
        .y(function(d) { return scaleY(d.y); });

    // now need to put both xValues and yValues in the same object to be able to send them to the "line" above in a method we will create below:
    var ourValues = [];

    newXValues.forEach( function (item, index) {     
        ourValues.push( { x: newXValues[index], y: newYValues[index] });   
    });

    // now puts the data into the line function
    // creates the line
    myLine
        .attr("class", "line")
        .datum(ourValues)
        .attr("stroke", function (d) {console.log("what is this"); console.log(d); return "red";})
        .transition().duration(1000)
        .attr("d", line);

    //appends the axis to what doesn't exist yet
    var xAxis = d3.svg.axis().scale(x).orient("bottom");

    var yAxis = d3.svg.axis().scale(y).orient("left");

    //Make the axis, have defined x and y at the top already
    x.domain([d3.min(ourValues, function(d) 
    { 
        return d.x; 
        
    }), d3.max(ourValues, function(d) 
    { 
        return d.x; 
    })]);


    // y goes from a negative to a positive value
    y.domain([d3.min(ourValues, function(d) 
    { 
        return d.y; 
        
    }), 

    d3.max(ourValues, function(d) 
    { 
        return d.y; 
        
    })]);

    //The axis and some labels - apparenly there comes some default values from 0.0-1.0 when the axis are added without binding them to some values
    xlabelaxis
        .transition().duration(1000)
        .call(xAxis)

    
    ylabelaxis
        .transition().duration(1000)
        .call(yAxis)
}

plotLine(xValues, yValues);
''')

# Now let’s make a template for the html string
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [10]:
js_text = js_text_template.substitute({'python_data': json.dumps(function_data),
                                       'graphdiv': 'graph-div'})
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))